In [7]:
resource = "../data/generated/"

In [8]:
import pandas as pd
from scipy.optimize import fmin_bfgs, basinhopping, differential_evolution
import scipy.optimize as opt 
from numpy import hstack, zeros, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from gmm import *

In [9]:
# Set the start and end dates of the analysis period
start_date = pd.to_datetime('1975-01-01')
end_date = pd.to_datetime('2008-04-30')

ts = pd.read_csv(resource + 'time_series_1.csv', parse_dates=['date'], index_col=['date'])
ts.index.freq = 'M'

# Set analysis period
ts1 = ts.loc[(ts.index >= start_date) & (ts.index <= end_date)]

In [10]:
bm_port = pd.read_csv(resource + f'bm_port.csv', parse_dates=['date'], index_col=['date'])
ts3 = pd.merge(ts1, bm_port, on='date', how='left')
macro_factors = ts3[['myp','ui','dsv','ats','sts','fx']].values
financial_factors = ts3[['ex_mkt','smb','hml','mom']].values
riskfree = ts3['rf'].values
portfolios = ts3[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
T,N = portfolios.shape
excessRet = portfolios - reshape(riskfree,(T,1))
K = np.size(macro_factors,1)

startingVals = zeros(6)
# startingVals = [
# -49.5247,
# -4.7737,
# 113.7766,
# 25.9245,
# 90.8717,
# 16.7917
# ]
# startingVals = [
# 1000,
# 1000,
# 1000,
# 1000,
# 1000,
# 1000
#  ]

Winv = eye(N)
args = (excessRet, macro_factors, Winv)
iteration = 0
lastValue = 0
functionCount = 0
# fit_gmm = fmin_bfgs(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)), callback=iter_print)
# fit_gmm = basinhopping(gmm_objective_b, startingVals, minimizer_kwargs={'args': args})
# fit_gmm = differential_evolution(gmm_objective_b, bounds=[(1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000)], args=args)
fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)),
                       method='BFGS', options={'gtol': 1e-10, 'maxiter': 1000})
# fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)),
#                        method='Nelder-Mead')
# fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)), callback=iter_print, method='L-BFGS-B')
fit_gmm

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.001689611555707051
        x: [-3.604e+01  1.696e+02  8.137e+01 -1.558e+02  3.320e+01
            -6.113e+00]
      nit: 139
      jac: [-1.150e-09  4.802e-10 -1.746e-10  1.019e-10  6.112e-10
             6.257e-10]
 hess_inv: [[ 1.755e+05  1.120e+05 ...  1.777e+05 -3.468e+04]
            [ 1.120e+05  8.404e+04 ...  9.996e+04 -2.345e+04]
            ...
            [ 1.777e+05  9.996e+04 ...  2.224e+05 -7.138e+04]
            [-3.468e+04 -2.345e+04 ... -7.138e+04  6.554e+04]]
     nfev: 1267
     njev: 181

In [11]:
import numpy as np
from statsmodels.sandbox.regression.gmm import GMM

class estimate_b(GMM):
    def momcond(self, params):
        fRets = self.exog
        pRets = self.instrument
        T,N = pRets.shape
        T,K = fRets.shape
        b = params
        b = squeeze(array(params)) 
        b = reshape(b,(K,1))
        sdf = ones((T,1)) - fRets @ b
        moments = pRets * kron(sdf,ones((1,N)))
        return moments

gmm_mod = estimate_b(endog=zeros(macro_factors.shape[0]), exog=macro_factors, instrument=excessRet)
gmm_fit8 = gmm_mod.fit(start_params=zeros(6), maxiter=1, inv_weights=eye(N), weights_method='hac', wargs={'maxlag':12}, optim_method='bfgs', optim_args={'gtol': 1e-10, 'maxiter': 1000})
gmm_fit8.summary()

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 138
         Function evaluations: 174
         Gradient evaluations: 174


<class 'statsmodels.iolib.summary.Summary'>
"""
                              estimate_b Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                     0.001690
Model:                     estimate_b   Prob (Hansen J):                  1.00
Method:                           GMM                                         
Date:                Wed, 10 May 2023                                         
Time:                        14:52:38                                         
No. Observations:                 400                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1           -36.0391     54.753     -0.658      0.510    -143.352      71.274
x2           169.5382    202.575      0.837      0.403    -227.501     566.577
x3            81.3747     56.940      1.429      0.153     -30.225     192.974
x4          -155.8005    140.894     -1.106      0.269    -431.947     120.346
x5            33.2021     86.896      0.382      0.702    -137.111     203.515
x6            -6.1122     34.160     -0.179      0.858     -73.065      60.841
==============================================================================
"""

In [12]:
# b = ([ -49.5247,
# -4.7737,
# 113.7766,
# 25.9245,
# 90.8717,
# 16.7917])
# b = ([-46.47,  4.675,  108.4,  4.868,  94.44,
                #   12.19])
# b = ([ -50.01969851,  22.80054891, 107.80604282, -27.68159424,
#         85.04568974,   4.42476452])
# b = ([ -30.11158833,  237.66483381,   88.98292253, -148.44282895,
#          52.95375349,  -13.72251736])
b = ([ -2.448e+01,  2.359e+02,  8.662e+01, -1.550e+02,  6.203e+01,
            -1.383e+01])
# b = ([ 20.24410124, -1.89313026, 16.9439695 , -2.33814035, -1.56179756,
#        -2.41001749])
J = gmm_objective_b(b, excessRet, macro_factors, eye(N))
J

0.01571008063240392